# Assinatura Digial RSA e de Chaum


### Grupo:

 - João Marques, a84684
 - Luis Bigas,   a76964

## Assinatura Digital RSA

In [22]:
def genRSAkey(bits):
    p = random_prime(2^(bits//2), 2^(bits//2-1), True)
    q = random_prime(2^(bits//2-1), 2^(bits//2-2),True)
    n = p*q
    m = (p-1)*(q-1)
    e = randint(2, m-1)
    while gcd(e, m) != 1:
        e = randint(2,m-1)
    d = power_mod(e, -1, m)
    PubKey = (n, e)
    Chave = (PubKey, d)
    return PubKey, d

In [23]:
PubKey,PrivKey = genRSAkey(1024)
n = PubKey[0]
e = PubKey[1]

## Assinatura

In [33]:
import hashlib
mOrig = "Nobody expects the Spanish Inquisition"

mEncoded = bytes(mOrig, 'utf-8')
HH = hashlib.sha256(mEncoded).hexdigest()
m=ZZ('0x'+HH)
m

36906822241210867014776754913837566155108892467980493944565876131413157803075

$s = m^d \mod n$

In [4]:
s = power_mod(m,PrivKey, n)
s

14678884494655761086981664602112026855426822853929644311143239098432749656096269501249247146495661556907541171525697248482810825128229994998852783976716397945555501893532923321950247234334254979367950002197130941623690482315454455747774111018197569163207320729203966446078431881749942135673266322428768012257

# Verificação

- PubKey (Chave Publica)
- mOrig  (Mensagem Original)
- s      (Assinatura da Mensagem)

In [5]:
import hashlib
mEncoded = bytes(mOrig, 'utf-8')
HH = hashlib.sha256(mEncoded).hexdigest()
ml=ZZ('0x'+HH)
ml

36906822241210867014776754913837566155108892467980493944565876131413157803075

$m = s^e \mod n$

In [6]:
m = power_mod(s,e,n)
m

36906822241210867014776754913837566155108892467980493944565876131413157803075

In [7]:
ml == m

True

In [8]:
def signRSA(PubKey,PrivKey,msg):
    mEncoded = bytes(msg, 'utf-8')
    HH = hashlib.sha256(mEncoded).hexdigest()
    m=ZZ('0x'+HH)
    s = power_mod(m,PrivKey, PubKey[0])
    return s

In [9]:
def verifyRSA(PubKey, msgOrig, msgSign):
    mEncoded = bytes(mOrig, 'utf-8')
    HH = hashlib.sha256(mEncoded).hexdigest()
    ml=ZZ('0x'+HH)
    m = power_mod(msgSign,PubKey[1],PubKey[0])

    return m==ml

In [10]:
m = "Nobody expects the Spanish Inquisition"

In [11]:
s = signRSA(PubKey,PrivKey,m)
s

14678884494655761086981664602112026855426822853929644311143239098432749656096269501249247146495661556907541171525697248482810825128229994998852783976716397945555501893532923321950247234334254979367950002197130941623690482315454455747774111018197569163207320729203966446078431881749942135673266322428768012257

In [12]:
verifyRSA(PubKey, m, s)

True

# Assinatura cega de Chaum

## Inicialização

$0 < k < n,$  $mdc(k,n) = 1$

In [13]:
def init(PubK):
    n,e=PubK
    k=randint(1,n-1)
    while gcd(k,n)!=1:
        k=randint(1,n-1)
    return k

## Blinding
$m'\equiv m(k)^e \mod n$

In [14]:
def blind(m,k,PubK):
    return (m * power_mod(k,PubK[1],PubK[0])) % PubK[0]

## Signing
$s' \equiv (m')^{d} \mod n$

In [15]:
def sign(blind,PubK,PrivK):
    return power_mod(blind,PrivK,PubK[0]) % PubK[0]

## Unblinding
$s \equiv k^{-1}s' \mod n$

In [16]:
def unblind(sign,k,PubK):
    return (power_mod(k,-1,PubK[0])*sign) % PubK[0]

## Verificação

In [17]:
import hashlib
mOrig="Nobody expects the Spanish Inquisition"
mEncoded = bytes(mOrig, 'utf-8')
HH = hashlib.sha256(mEncoded).hexdigest()
m=ZZ('0x'+HH)
m

36906822241210867014776754913837566155108892467980493944565876131413157803075

In [18]:
PubK, PrivK=genRSAkey(2048)
print("n: ", PubK[0], "\ne: ", PubK[1], "\nd: ", PrivK)

n:  1380095391656916560331467097851016706165149106490383722773015676361632498100848607506862017628920773681068408216442891511062041940304546855054798808735478973998049463085260497056973427674278495240046061114518056325925509618732108009525101018093790444228908488056386641362229187806514056131176469151435569967096614531214354792690884184452605449190884820405780383767181218345815704115813459477857513434451591972220485797087679953940494730085215416627006284578852629343252029733294099846658575094627363958851720780061066272873053304339647481727006246248362173615995518913214685655489072548405250658358305670013806136423 
e:  13488427733532768703164099176055326012284724796312732466683435776581518136346006112468276801619362295507564893060829629666061825723815553489380320608586060448505486735366812120451427878038253397043080154353166302156120727523459441755684103510005929632319705037555666054149038748637387325956408952052612837361978685755300684173443102593058471889961418771140624465331434858163

In [19]:
k = init(PubK)
blinded = blind(m,k, PubK)
signed = sign(blinded, PubK, PrivK)
unblinded = unblind(signed, k, PubK)
print("k: ", k, "\n")
print("m': ", blinded, "\n")
print("s': ", signed, "\n")
print("s : ", unblinded, "\n")

k:  810024230141536697205922351071822651532741828096490646047911650579821005578605262323422440653548160257902902336475778948212965566081753923164060472320933494538108719444840336265454826423641264306261112841661135946663060306813735398905149616963786983768072455151622935456702400074005599147422841591074075544512991761370237147676125605503662130908622723929501713777770224067588147995440860381279426017727323303713939028362095775737018063383383792738349921379268973944811367728809174840391018582379146519554452992424194910279194368337547864530714301893530938339420558786254654695383928349074391739732090203382469295445 

m':  1044548107877649078235007887290177398587085283933554365435772088329242914308315488655774253931354393863995949658787509627706766927110421809652222904940869268141599644788009539664418213885662489207671422524612829138761099001516108493386715776914943955365714166769665452296956107034994741867787837932107467843666499497833270383862825663902208427746620804490362661109723712438

In [20]:
unblinded == power_mod(m,PrivK,PubK[0])

True